<a href="https://colab.research.google.com/github/duonghieu7104/Trend-Analysis-Aviation-Industry-US/blob/main/notebook/Market_Share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [187]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count, mean, stddev, sum
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit

In [188]:
spark = SparkSession.builder \
    .appName("Airline Market Share Prediction") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [189]:
import kagglehub
path = kagglehub.dataset_download("bhavikjikadara/us-airline-flight-routes-and-fares-1993-2024")
print(f"Dataset downloaded to: {path}")

Dataset downloaded to: C:\Users\PC\.cache\kagglehub\datasets\bhavikjikadara\us-airline-flight-routes-and-fares-1993-2024\versions\1


In [190]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("quote", '"')           # rất quan trọng: nhận diện text trong ngoặc kép
    .option("escape", '"')          # escape ký tự "
    .option("multiLine", "true")    # xử lý text có xuống dòng
    .option("mode", "PERMISSIVE")   # không drop dòng lỗi
    .option("nullValue", "")
    .option("nanValue", "NaN")
    .option("emptyValue", "")
    .csv(f"{path}/*.csv")
)

In [191]:
df.printSchema()
print(f"\nSố dòng: {df.count()}")
print(f"Số cột: {len(df.columns)}")

root
 |-- tbl: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- citymarketid_1: integer (nullable = true)
 |-- citymarketid_2: integer (nullable = true)
 |-- city1: string (nullable = true)
 |-- city2: string (nullable = true)
 |-- airportid_1: integer (nullable = true)
 |-- airportid_2: integer (nullable = true)
 |-- airport_1: string (nullable = true)
 |-- airport_2: string (nullable = true)
 |-- nsmiles: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- carrier_lg: string (nullable = true)
 |-- large_ms: double (nullable = true)
 |-- fare_lg: double (nullable = true)
 |-- carrier_low: string (nullable = true)
 |-- lf_ms: double (nullable = true)
 |-- fare_low: double (nullable = true)
 |-- Geocoded_City1: string (nullable = true)
 |-- Geocoded_City2: string (nullable = true)
 |-- tbl1apk: string (nullable = true)


Số dòng: 245955
Số cột: 23


In [192]:
df.show(5)


+-------+----+-------+--------------+--------------+--------------------+--------------------+-----------+-----------+---------+---------+-------+----------+------+----------+--------+-------+-----------+------+--------+--------------+--------------+--------------------+
|    tbl|Year|quarter|citymarketid_1|citymarketid_2|               city1|               city2|airportid_1|airportid_2|airport_1|airport_2|nsmiles|passengers|  fare|carrier_lg|large_ms|fare_lg|carrier_low| lf_ms|fare_low|Geocoded_City1|Geocoded_City2|             tbl1apk|
+-------+----+-------+--------------+--------------+--------------------+--------------------+-----------+-----------+---------+---------+-------+----------+------+----------+--------+-------+-----------+------+--------+--------------+--------------+--------------------+
|Table1a|2021|      3|         30135|         33195|Allentown/Bethleh...|Tampa, FL (Metrop...|      10135|      14112|      ABE|      PIE|    970|       180| 81.43|        G4|     1.0|

In [193]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+---+----+-------+--------------+--------------+-----+-----+-----------+-----------+---------+---------+-------+----------+----+----------+--------+-------+-----------+-----+--------+--------------+--------------+-------+
|tbl|Year|quarter|citymarketid_1|citymarketid_2|city1|city2|airportid_1|airportid_2|airport_1|airport_2|nsmiles|passengers|fare|carrier_lg|large_ms|fare_lg|carrier_low|lf_ms|fare_low|Geocoded_City1|Geocoded_City2|tbl1apk|
+---+----+-------+--------------+--------------+-----+-----+-----------+-----------+---------+---------+-------+----------+----+----------+--------+-------+-----------+-----+--------+--------------+--------------+-------+
|  0|   0|      0|             0|             0|    0|    0|          0|          0|        0|        0|      0|         0|   0|      1540|    1540|   1540|       1612| 1612|    1612|         39206|         39206|      0|
+---+----+-------+--------------+--------------+-----+-----+-----------+-----------+---------+---------+-------+

In [194]:
df.select('passengers', 'fare', 'large_ms', 'nsmiles', 'lf_ms').describe().show()

+-------+------------------+-----------------+------------------+-----------------+-------------------+
|summary|        passengers|             fare|          large_ms|          nsmiles|              lf_ms|
+-------+------------------+-----------------+------------------+-----------------+-------------------+
|  count|            245955|           245955|            244415|           245955|             244343|
|   mean| 299.4767945355858|218.9795867943276|0.6652516330831235|1189.812319326706|0.45043751243130975|
| stddev|511.38948561649295|82.37248581123856|0.2246346557290856|703.1434716080329| 0.3326690289129114|
|    min|                 0|             50.0|            0.0038|              109|               0.01|
|    max|              8301|           3377.0|               1.0|             2724|                1.0|
+-------+------------------+-----------------+------------------+-----------------+-------------------+



In [195]:
df_clean = df.filter(col('large_ms').isNotNull())
print(f"\nKích thước dữ liệu sau khi loại bỏ dòng lỗi: {df_clean.count()} rows, {len(df_clean.columns)} columns")


Kích thước dữ liệu sau khi loại bỏ dòng lỗi: 244415 rows, 23 columns


In [196]:
df_clean = df_clean.filter((col('large_ms') >= 0) & (col('large_ms') <= 1))
print(f"Số dòng sau khi làm sạch: {df_clean.count()}")

Số dòng sau khi làm sạch: 244415


In [197]:
df_clean = df_clean.filter(
    (col('passengers') > 0) & 
    (col('fare') > 0) & 
    (col('nsmiles') > 0)
)

In [198]:
df_clean = df_clean.dropna(subset=['carrier_low'])


In [199]:

df_clean.select([
    sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in df_clean.columns
]).show()

+---------+----------+-------------+--------------------+--------------------+-----------+-----------+-----------------+-----------------+---------------+---------------+-------------+----------------+----------+----------------+--------------+-------------+-----------------+-----------+--------------+--------------------+--------------------+-------------+
|tbl_nulls|Year_nulls|quarter_nulls|citymarketid_1_nulls|citymarketid_2_nulls|city1_nulls|city2_nulls|airportid_1_nulls|airportid_2_nulls|airport_1_nulls|airport_2_nulls|nsmiles_nulls|passengers_nulls|fare_nulls|carrier_lg_nulls|large_ms_nulls|fare_lg_nulls|carrier_low_nulls|lf_ms_nulls|fare_low_nulls|Geocoded_City1_nulls|Geocoded_City2_nulls|tbl1apk_nulls|
+---------+----------+-------------+--------------------+--------------------+-----------+-----------+-----------------+-----------------+---------------+---------------+-------------+----------------+----------+----------------+--------------+-------------+-----------------+----

In [200]:
print(f"Số dòng còn lại: {df_clean.count()}")


Số dòng còn lại: 237985


In [201]:
df_features = df_clean.withColumn(
    'revenue', col('passengers') * col('fare')
).withColumn(
    'fare_per_mile', col('fare') / col('nsmiles')
).withColumn(
    'fare_difference', col('fare') - col('fare_low')
).withColumn(
    'is_largest_cheapest', 
    when(col('carrier_lg') == col('carrier_low'), 1).otherwise(0)
)

In [202]:
numeric_features = [
    'passengers', 'fare', 'nsmiles', 'lf_ms',
    'fare_low', 'revenue', 'fare_per_mile', 'fare_difference',
    'Year', 'quarter'
]

In [203]:
categorical_features = ['carrier_lg', 'carrier_low']

In [204]:
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_indexed", handleInvalid="keep")
    for col in categorical_features
]

In [205]:
df_features.printSchema()

root
 |-- tbl: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- citymarketid_1: integer (nullable = true)
 |-- citymarketid_2: integer (nullable = true)
 |-- city1: string (nullable = true)
 |-- city2: string (nullable = true)
 |-- airportid_1: integer (nullable = true)
 |-- airportid_2: integer (nullable = true)
 |-- airport_1: string (nullable = true)
 |-- airport_2: string (nullable = true)
 |-- nsmiles: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- carrier_lg: string (nullable = true)
 |-- large_ms: double (nullable = true)
 |-- fare_lg: double (nullable = true)
 |-- carrier_low: string (nullable = true)
 |-- lf_ms: double (nullable = true)
 |-- fare_low: double (nullable = true)
 |-- Geocoded_City1: string (nullable = true)
 |-- Geocoded_City2: string (nullable = true)
 |-- tbl1apk: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- fare_per_m

In [206]:
feature_cols = numeric_features + [col+"_indexed" for col in categorical_features]
feature_cols.append('is_largest_cheapest')

In [207]:
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw"
)

In [208]:
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

In [209]:
train_data, test_data = df_features.randomSplit([0.8, 0.2], seed=42)

print(f"\n=== CHIA DỮ LIỆU ===")
print(f"Số dòng train: {train_data.count()}")
print(f"Số dòng test: {test_data.count()}")


=== CHIA DỮ LIỆU ===
Số dòng train: 190123
Số dòng test: 47862


In [210]:
lr = LinearRegression(
    featuresCol="features",
    labelCol="large_ms",
    maxIter=100,
    regParam=0.1,
    elasticNetParam=0.5
)

In [211]:
pipeline_lr = Pipeline(stages=indexers + [assembler, scaler, lr])
model_lr = pipeline_lr.fit(train_data)
predictions_lr = model_lr.transform(test_data)

In [212]:
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="large_ms",
    numTrees=100,
    maxDepth=10,
    seed=42
)

In [213]:
pipeline_rf = Pipeline(stages=indexers + [assembler, scaler, rf])
model_rf = pipeline_rf.fit(train_data)
predictions_rf = model_rf.transform(test_data)

In [214]:
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="large_ms",
    maxIter=100,
    maxDepth=5,
    seed=42
)

In [215]:

pipeline_gbt = Pipeline(stages=indexers + [assembler, scaler, gbt])
model_gbt = pipeline_gbt.fit(train_data)
predictions_gbt = model_gbt.transform(test_data)

In [216]:
evaluator_rmse = RegressionEvaluator(
    labelCol="large_ms",
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_r2 = RegressionEvaluator(
    labelCol="large_ms",
    predictionCol="prediction",
    metricName="r2"
)

evaluator_mae = RegressionEvaluator(
    labelCol="large_ms",
    predictionCol="prediction",
    metricName="mae"
)

In [217]:
print("\nLinear Regression:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_lr):.4f}")
print(f"  R2: {evaluator_r2.evaluate(predictions_lr):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_lr):.4f}")

print("\nRandom Forest:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_rf):.4f}")
print(f"  R2: {evaluator_r2.evaluate(predictions_rf):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_rf):.4f}")

print("\nGradient Boosted Trees:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_gbt):.4f}")
print(f"  R2: {evaluator_r2.evaluate(predictions_gbt):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_gbt):.4f}")


Linear Regression:
  RMSE: 0.1943
  R2: 0.2376
  MAE: 0.1699

Random Forest:
  RMSE: 0.0840
  R2: 0.8573
  MAE: 0.0554

Gradient Boosted Trees:
  RMSE: 0.0765
  R2: 0.8818
  MAE: 0.0481


In [218]:
rf_model = model_rf.stages[-1]
feature_importance = rf_model.featureImportances.toArray()

feature_names = feature_cols
importance_df = spark.createDataFrame(
    [(name, float(imp)) for name, imp in zip(feature_names, feature_importance)],
    ['feature', 'importance']
).orderBy(col('importance').desc())

importance_df.show(20)

+-------------------+--------------------+
|            feature|          importance|
+-------------------+--------------------+
|              lf_ms|  0.6691844630064311|
|    fare_difference| 0.13569947965126553|
|            nsmiles| 0.06421050930453445|
|      fare_per_mile| 0.03515753599119153|
|is_largest_cheapest|0.034181782479342816|
|            revenue| 0.01794006182416434|
|         passengers|0.016684549979397834|
| carrier_lg_indexed|0.010819914920363834|
|               Year|0.006604698071603944|
|carrier_low_indexed|0.004857337121019847|
|               fare|0.002854490995978...|
|           fare_low|0.001720933992446...|
|            quarter|8.424266225933034E-5|
+-------------------+--------------------+



In [219]:
predictions_rf.select(
    'large_ms', 'prediction', 
    'passengers', 'fare', 'nsmiles', 'carrier_lg'
).show(10)

+--------+------------------+----------+------+-------+----------+
|large_ms|        prediction|passengers|  fare|nsmiles|carrier_lg|
+--------+------------------+----------+------+-------+----------+
|    0.85|0.8534900135796164|       133|275.63|    654|        UA|
|    0.37|0.5309484391287608|         4|267.08|   1974|        US|
|    0.26|0.4312600977159158|        68|258.75|   1670|        DL|
|    0.83|0.8495292097959413|        57|139.36|   1123|        WN|
|    0.66| 0.646960262991727|         4|223.75|   1861|        NW|
|     0.8|0.7667079022924445|         7| 326.4|   1861|        DL|
|    0.94|0.9354262355257376|       888| 86.67|    677|        WN|
|    0.53|0.4827445549495522|        23|289.51|   1504|        UA|
|     0.5|0.4973493884381909|        26|222.54|    963|        UA|
|    0.44|0.5055744906018458|        71|204.62|    963|        UA|
+--------+------------------+----------+------+-------+----------+
only showing top 10 rows



In [221]:
lr = LinearRegression(featuresCol="features", labelCol="large_ms", maxIter=100)

In [222]:
paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, 0.01, 0.1, 0.5, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr.maxIter, [50, 100, 200]) \
    .build()

In [223]:
# Cross-validator
cv_lr = CrossValidator(
    estimator=Pipeline(stages=indexers + [assembler, scaler, lr]),
    estimatorParamMaps=paramGrid_lr,
    evaluator=evaluator_rmse,
    numFolds=3,  # 3-fold cross-validation
    parallelism=2,
    seed=42
)

In [224]:
cv_model_lr = cv_lr.fit(train_data)

In [225]:
best_lr_model = cv_model_lr.bestModel.stages[-1]
print("Best parameters cho Linear Regression:")
print(f"  regParam: {best_lr_model.getRegParam()}")
print(f"  elasticNetParam: {best_lr_model.getElasticNetParam()}")
print(f"  maxIter: {best_lr_model.getMaxIter()}\n")

Best parameters cho Linear Regression:
  regParam: 0.0
  elasticNetParam: 0.0
  maxIter: 50



In [226]:
predictions_lr = cv_model_lr.transform(test_data)
print("Kết quả trên Test Set:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_lr):.4f}")
print(f"  R²: {evaluator_r2.evaluate(predictions_lr):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_lr):.4f}\n")

Kết quả trên Test Set:
  RMSE: 0.1721
  R²: 0.4013
  MAE: 0.1381



In [227]:
# Cross-validation scores
cv_scores_lr = cv_model_lr.avgMetrics
print("Cross-Validation Scores (RMSE):")
for idx, score in enumerate(cv_scores_lr[:5]):  # Top 5
    print(f"  Config {idx+1}: {score:.4f}")


Cross-Validation Scores (RMSE):
  Config 1: 0.1725
  Config 2: 0.1725
  Config 3: 0.1725
  Config 4: 0.1725
  Config 5: 0.1725


In [228]:
rf = RandomForestRegressor(featuresCol="features", labelCol="large_ms", seed=42)
paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 150]) \
    .addGrid(rf.maxDepth, [5, 10, 15, 20]) \
    .addGrid(rf.minInstancesPerNode, [1, 5, 10]) \
    .addGrid(rf.maxBins, [32, 64]) \
    .build()

In [229]:
# TrainValidationSplit (nhanh hơn CrossValidator cho RF)
tvs_rf = TrainValidationSplit(
    estimator=Pipeline(stages=indexers + [assembler, scaler, rf]),
    estimatorParamMaps=paramGrid_rf,
    evaluator=evaluator_rmse,
    trainRatio=0.8,  # 80% train, 20% validation
    parallelism=2,
    seed=42
)

In [230]:
tvs_model_rf = tvs_rf.fit(train_data)

In [231]:
# Best parameters
best_rf_model = tvs_model_rf.bestModel.stages[-1]
print("Best parameters cho Random Forest:")
print(f"  numTrees: {best_rf_model.getNumTrees}")
print(f"  maxDepth: {best_rf_model.getMaxDepth()}")
print(f"  minInstancesPerNode: {best_rf_model.getMinInstancesPerNode()}")
print(f"  maxBins: {best_rf_model.getMaxBins()}\n")

Best parameters cho Random Forest:
  numTrees: 150
  maxDepth: 20
  minInstancesPerNode: 1
  maxBins: 64



In [232]:
# Đánh giá trên test set
predictions_rf = tvs_model_rf.transform(test_data)
print("Kết quả trên Test Set:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_rf):.4f}")
print(f"  R²: {evaluator_r2.evaluate(predictions_rf):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_rf):.4f}\n")

Kết quả trên Test Set:
  RMSE: 0.0670
  R²: 0.9092
  MAE: 0.0394



In [233]:
# GBT với hyperparameter tuning
gbt = GBTRegressor(featuresCol="features", labelCol="large_ms", seed=42)

# Param grid cho GBT
paramGrid_gbt = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [50, 100, 150]) \
    .addGrid(gbt.maxDepth, [3, 5, 7]) \
    .addGrid(gbt.stepSize, [0.01, 0.1, 0.2]) \
    .addGrid(gbt.subsamplingRate, [0.8, 1.0]) \
    .build()

In [234]:
# TrainValidationSplit cho GBT
tvs_gbt = TrainValidationSplit(
    estimator=Pipeline(stages=indexers + [assembler, scaler, gbt]),
    estimatorParamMaps=paramGrid_gbt,
    evaluator=evaluator_rmse,
    trainRatio=0.8,
    parallelism=2,
    seed=42
)

In [235]:
tvs_model_gbt = tvs_gbt.fit(train_data)


In [236]:
# Best parameters
best_gbt_model = tvs_model_gbt.bestModel.stages[-1]
print("Best parameters cho GBT:")
print(f"  maxIter: {best_gbt_model.getMaxIter()}")
print(f"  maxDepth: {best_gbt_model.getMaxDepth()}")
print(f"  stepSize: {best_gbt_model.getStepSize()}")
print(f"  subsamplingRate: {best_gbt_model.getSubsamplingRate()}\n")

Best parameters cho GBT:
  maxIter: 150
  maxDepth: 7
  stepSize: 0.1
  subsamplingRate: 1.0



In [237]:
# Đánh giá trên test set
predictions_gbt = tvs_model_gbt.transform(test_data)
print("Kết quả trên Test Set:")
print(f"  RMSE: {evaluator_rmse.evaluate(predictions_gbt):.4f}")
print(f"  R²: {evaluator_r2.evaluate(predictions_gbt):.4f}")
print(f"  MAE: {evaluator_mae.evaluate(predictions_gbt):.4f}\n")

Kết quả trên Test Set:
  RMSE: 0.0695
  R²: 0.9024
  MAE: 0.0429

